In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [2]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'MDE'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 0

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

### Loading COSTUMERS INFO

In [3]:
#//--------------------------------------
#//Ruta de data // KAM's costumers to delete
#//KAMs customers me lo manda Emiliano Lomeli (SE ACTUALIZAN DE UN SCRIPT MES A MES)
users_eliminar_kam = pd.read_csv("SD/import_data/segments/KAMs_JUL.csv")['costumer_id'].unique().tolist()
print(len(users_eliminar_kam))

2595


In [4]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [5]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",48322,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


### PARAMETERS // SOME FUNCTIONS - GET READY TO PROCESSING

In [6]:
active_skus = frida_products[frida_products.status_frida == 'on']["sku_id"].unique()
data_ventas = data_ventas[data_ventas.sku_id.isin(active_skus)]

data_ventas.shape

(71563, 19)

In [7]:
data_ventas = data_ventas.groupby(by=['region_code','customer_id','category','subcat','padre_sku_id']).agg({"name":pd.Series.unique, "gmv_usd":np.sum, "discount_applied":np.sum, "cant": np.sum, "order_id":pd.Series.nunique}).reset_index()

In [8]:
frida_products['sku_id'] = frida_products['sku_id'].astype(str)

In [9]:
frida_products_clean = frida_products[(frida_products.status_frida == 'on') & (frida_products["ciudad::multiFilter"] == city_code)].groupby(by= ['id_tarjeta']).apply(lambda x: '#'.join(x["sku_id"].unique())).reset_index()
frida_products_clean = frida_products_clean.rename(columns={0:'skus'})

In [10]:
data_ventas_df = pd.merge(data_ventas,frida_products_clean, left_on='padre_sku_id', right_on='id_tarjeta')#.drop(columns = ['id_tarjeta'])

In [11]:
def skus_offer(skus):
    
    skus_aux = [frida_products_clean.loc[frida_products_clean['skus'].str.contains(sk, case=False)]['skus'].to_list()[0] for sku in skus for sk in sku]
    
    a=[]*len(skus)
    contador=0
    contador2=0
    for i,sku in enumerate(skus):
        contador+= len(sku)
        a.append(skus_aux[contador2:contador])
        contador2+= len(sku)
        
    return a

In [12]:
def list_users(data_ventas, rangos):
    """This function select the customer_ids of the clients who are between the range of gmv_usd"""
    #data_ventas = data_ventas[data_ventas.subcat == subcat]
    
    total_users = data_ventas.customer_id.unique().tolist()
    
    print(f'Total users: {len(total_users)}')
    
    inferior, superior = np.percentile(data_ventas['gmv_usd'],rangos) #rango intercuartil entre los skus a tomar
    
    users = data_ventas[(data_ventas.gmv_usd>=inferior) & (data_ventas.gmv_usd<=superior)]['customer_id'].unique()
    print(f'Total users: {len(users)}')
    
    return users

In [13]:
def list_users(data_ventas, subcat, rangos):
    """This function select the customer_ids of the clients who are between the range of gmv_usd"""
    data_ventas = data_ventas[data_ventas.subcat == subcat]
    
    total_users = data_ventas.customer_id.unique().tolist()
    
    print(f'Total users: {len(total_users)}')
    
    inferior, superior = np.percentile(data_ventas['gmv_usd'],rangos) #rango intercuartil entre los skus a tomar
    
    users = data_ventas[(data_ventas.gmv_usd>=inferior) & (data_ventas.gmv_usd<=superior)]['customer_id'].unique()
    print(f'Total users: {len(users)}')
    
    return users

# Customize Offers

### Load the information in the next block

In [14]:
"""Here we should complete our data for each SH, important if we want to submit a subcategory SH offer, in SKUS we should put the skus into a list as this [[1,2,3]]"""
offers_name = ['Aceite']
skus = [['570403']
        #,['568617']
        #,['588961','588963','588965','508901','242095']
        #,['604761','604769','604765','604435','604433','604739','604755','604745','604771','604751','604757','604741','604743','604749','604763']
        #,['71950','212521','327839','57791','57797','57803','57817','71944']
        ] #it should be the sku in string
discount = [10]
max_uses = [10]
max_orders = [2]
#////////////
rango_IQ = [[0,90]]
users = []

In [15]:
for j, sku in enumerate(skus):
    range = rango_IQ[j]
    print(range)
    
    if len(sku) == 1:
        x = []
        aux_data = data_ventas_df.loc[data_ventas_df['skus'].str.contains(sku[0], case=False)]
        
        sub = data_ventas_df.loc[data_ventas_df['skus'].str.contains(sku[0], case=False)].subcat.unique()[0]
        print(sub)
        
        x.append(list_users(data_ventas_df,sub,range))
    else:
        x = []
        y = []
        for i in sku:
            aux_data = data_ventas_df.loc[data_ventas_df['skus'].str.contains(i, case=False)]
            y.append(aux_data)
        aux_data_all = pd.concat(y)
        aux_data_all = aux_data_all.groupby(by=['region_code','customer_id','category','subcat']).agg({"gmv_usd":np.sum, "discount_applied":np.sum, "cant": np.sum, "order_id":pd.Series.nunique}).reset_index()
        
        sub = aux_data_all.subcat.unique()[0]
        print(sub)
        
        x.append(list_users(data_ventas_df,sub,range))
    users.append(x)

[0, 90]
Aceites & Grasas
Total users: 2245
Total users: 2046


# Generate CSV for Offers

In [16]:
global_segment_list = [] #//for procesing
r= {}

for i,offer in enumerate(offers_name):
    id_list = users[i][0].tolist()
    test = random.sample(list(id_list), int(len(id_list)*0.85))
    control = [value for value in id_list if value not in test]
    d = {'name':"SH_"+todays_date+"_"+city_code+"_"+offer, 'customersIds': list(test)}

    r['CONTROL_'+"SH_"+todays_date+"_"+city_code+"_"+offer] = control
    r['TEST_'+"SH_"+todays_date+"_"+city_code+"_"+offer] = test
    
    global_segment_list.append(d) 

df_test_control = pd.DataFrame.from_dict(r, orient='index').T

In [17]:
csv_file = {} 
csv_file['offer_name'] = ["SH_"+todays_date+"_"+city_code+"_"+x for x in offers_name]
csv_file['offer_description'] = None
csv_file['discount'] = discount
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=dias)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(dias+7))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = max_uses
csv_file['max_uses_per_customer'] = max_orders
csv_file['customer_segment_id'] = [1]*len(offers_name) #[None]
csv_file['sku_id'] = ["#".join(s) for s in skus_offer(skus)]

global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [18]:
import requests
import json

url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [19]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [20]:
for i in global_offer_csv.offer_name:
    for j,z in sgt:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j
            break

In [21]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_2023-01-10_MDE_Aceite,None,10,true,10-01-2023,17-01-2023,10,2,114551,570403


In [22]:
#//--------------------------------------------
#//Exporting single file data
with open(f'SD/export_data/Below_Media/{city_code}/{todays_date}_spend_{city_code}_All_SH.txt', 'w') as outfile:
        json.dump(global_segment_list, outfile)

In [23]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/Below_Media/{city_code}/{todays_date}_OFFER_csv_{city_code}_All_SH.csv', index = False)
df_test_control.to_csv(f'SD/export_data/Below_Media/{city_code}/{todays_date}_CLIENTS_csv_{city_code}_All_SH.csv', index = False)